In [ ]:
from fabrictestbed_extensions.fablib.fablib 
import FablibManager as fablib_manager
import time
import psutil

fablib = fablib_manager()

slice_name = 'priority_path_allocation'
slice = fablib.get_slice(name=slice_name)

MIN_TRAFFIC_THRESHOLD = 1  
CHECK_INTERVAL = 60  

def check_traffic(interface_name):
    try:
        print(f"Checking traffic on interface: {interface_name}")
        net_io = psutil.net_io_counters(pernic=True)
        if interface_name in net_io:
            bytes_sent = net_io[interface_name].bytes_sent
            bytes_recv = net_io[interface_name].bytes_recv
            traffic_mbps = (bytes_sent + bytes_recv) * 8 / (1024 * 1024)
            print(f"Traffic on {interface_name}: {traffic_mbps} Mbps")
            return traffic_mbps
        else:
            print(f"Interface {interface_name} not found in net_io counters.")
    except Exception as e:
        print(f"Error checking traffic on {interface_name}: {e}")
    return 0

def shutdown_if_no_traffic(node_name, interface_name):
    print(f"Checking if node {node_name} interface {interface_name} should be shut down...")
    traffic = check_traffic(interface_name)
    if traffic < MIN_TRAFFIC_THRESHOLD:
        print(f"Traffic on {interface_name} is below threshold ({traffic} Mbps), attempting to shut down {node_name}.")
        try:
            node = slice.get_node(name=node_name)
            interface = next((iface for iface in node.get_interfaces() if iface.get_name() == interface_name), None)
            if interface:
                print(f"Shutting down interface {interface_name} on node {node_name}.")
                node.execute(f"sudo ip link set {interface_name} down")
                print(f"Node {node_name} interface {interface_name} has been shut down.")
            else:
                print(f"Interface {interface_name} not found on node {node_name}, skipping shutdown.")
        except Exception as e:
            print(f"Failed to shut down {node_name}: {e}")

def bring_node_up(node_name, interface_name):
    print(f"Attempting to bring up node {node_name} interface {interface_name}...")
    try:
        node = slice.get_node(name=node_name)
        interface = next((iface for iface in node.get_interfaces() if iface.get_name() == interface_name), None)
        if interface:
            print(f"Bringing up interface {interface_name} on node {node_name}.")
            node.execute(f"sudo ip link set {interface_name} up")
            print(f"Node {node_name} interface {interface_name} has been brought back up.")
        else:
            print(f"Interface {interface_name} not found on node {node_name}, skipping bring up.")
    except Exception as e:
        print(f"Failed to bring up {node_name}: {e}")

def get_interfaces_to_monitor():
    nodes_to_monitor = []
    print("Retrieving interfaces to monitor...")
    for node in slice.get_nodes():
        try:
            print(f"Retrieving interfaces for node: {node.get_name()}")
            interfaces = node.get_interfaces()
            for interface in interfaces:
                print(f"Found interface {interface.get_name()} on node {node.get_name()}")
                nodes_to_monitor.append({"name": node.get_name(), "interface": interface.get_name()})
        except Exception as e:
            print(f"Failed to retrieve interfaces for node {node.get_name()}: {e}")
    return nodes_to_monitor

def monitor_and_manage():
    print("Starting to monitor and manage interfaces...")
    nodes_to_monitor = get_interfaces_to_monitor()
    print(f"Monitoring the following interfaces: {nodes_to_monitor}")
    while True:
        for node_info in nodes_to_monitor:
            node_name = node_info["name"]
            interface_name = node_info["interface"]
            shutdown_if_no_traffic(node_name, interface_name)
        time.sleep(CHECK_INTERVAL)

if __name__ == "__main__":
    try:
        print("Starting energy-efficient routing monitor...")
        monitor_and_manage()
    except KeyboardInterrupt:
        print("Stopping energy-efficient routing monitor.")


Starting energy-efficient routing monitor...
Starting to monitor and manage interfaces...
Retrieving interfaces to monitor...
Retrieving interfaces for node: client
Found interface client-clientToSwitchMed-p1 on node client
Found interface client-clientToSwitchLow-p1 on node client
Found interface client-clientToSwitchHigh-p1 on node client
Retrieving interfaces for node: server
Retrieving interfaces for node: switch_high_priority
Found interface switch_high_priority-switchToClientHigh-p1 on node switch_high_priority
Retrieving interfaces for node: switch_medium_priority
Found interface switch_medium_priority-switchMed-p1 on node switch_medium_priority
Retrieving interfaces for node: switch_low_priority
Found interface switch_low_priority-switchLow-p1 on node switch_low_priority
Monitoring the following interfaces: [{'name': 'client', 'interface': 'client-clientToSwitchMed-p1'}, {'name': 'client', 'interface': 'client-clientToSwitchLow-p1'}, {'name': 'client', 'interface': 'client-clie